In [ ]:
#IMPORTS & LOGS
import os
import sys
import random
import time
import copy
import gc
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset, Sampler
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
import optuna

#Settings
CONFIG = {
    'seed': 42,
    'img_size': 224,
    'num_workers': 2,
    'batch_size': 720,     
    'n_trials': 20,        
    'epochs_per_trial': 6, 
    'log_file': 'training_log.txt'
}

#Logs
def log(message):
    print(message)
    with open(CONFIG['log_file'], "a") as f:
        f.write(message + "\n")

#Clears old existing log
if os.path.exists(CONFIG['log_file']):
    os.remove(CONFIG['log_file'])

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    log(f" Seed set at {seed}")

set_seed(CONFIG['seed'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log(f" Device: {device} | GPU Count: {torch.cuda.device_count()}")
if torch.cuda.device_count() > 1:
    log(f"Using Multi-GPU: Effective Batch for GPU ~= {CONFIG['batch_size'] // torch.cuda.device_count()}")

In [ ]:
#DATASET & SAMPLER

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(CONFIG['img_size'], scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

class BookCoverDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, class_to_idx=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(self.df['Category'].unique())
        self.class_to_idx = class_to_idx if class_to_idx else {cls: i for i, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, str(row['Filename']))
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (CONFIG['img_size'], CONFIG['img_size']))
        
        label = self.class_to_idx[row['Category']]
        if self.transform:
            image = self.transform(image)
        return image, label

class BalancedBatchSampler(Sampler):
    def __init__(self, dataset_df, batch_size):
        self.df = dataset_df
        self.batch_size = batch_size
        self.classes = sorted(self.df['Category'].unique())
        self.n_classes = len(self.classes)
        
        # Check batch division
        if self.batch_size % self.n_classes != 0:
             raise ValueError(f"Batch size {self.batch_size} non divisible by {self.n_classes} classes.")

        self.samples_per_class = self.batch_size // self.n_classes
        
        self.class_indices = {c: [] for c in self.classes}
        for idx, row in self.df.iterrows():
            self.class_indices[row['Category']].append(idx)
            
        self.min_samples = min(len(v) for v in self.class_indices.values())
        self.n_batches = self.min_samples // self.samples_per_class
        
        log(f"Sampler: {self.samples_per_class} img/class x {self.n_classes} classes = {self.batch_size} BS.")

    def __iter__(self):
        for c in self.classes:
            np.random.shuffle(self.class_indices[c])
            
        for i in range(self.n_batches):
            batch = []
            for c in self.classes:
                batch.extend(self.class_indices[c][i*self.samples_per_class : (i+1)*self.samples_per_class])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return self.n_batches

#Preparing data
# File Search
INPUT_DIR = '/kaggle/input'
csv_train_path = None
img_dir = None

for root, dirs, files in os.walk(INPUT_DIR):
    if "book30-listing-train.csv" in files:
        csv_train_path = os.path.join(root, "book30-listing-train.csv")
    if "224x224" in dirs:
        img_dir = os.path.join(root, "224x224")

if not csv_train_path: raise FileNotFoundError("CSV Train not found!")

#Load data and split
full_df = pd.read_csv(csv_train_path, sep=';', encoding='ISO-8859-1', on_bad_lines='warn')
train_df, val_df = train_test_split(full_df, test_size=0.2, stratify=full_df['Category'], random_state=CONFIG['seed'])

#Mandatory index reset for the sampler
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_ds = BookCoverDataset(train_df, img_dir, transform=data_transforms['train'])
val_ds = BookCoverDataset(val_df, img_dir, transform=data_transforms['val'], class_to_idx=train_ds.class_to_idx)

log("Data loaded and ready")

In [ ]:
#OPTUNA SEARCH

def objective(trial):
    #Suggested hyperparameters
    lr_backbone = trial.suggest_float("lr_backbone", 1e-6, 1e-4, log=True)
    lr_head = trial.suggest_float("lr_head", 1e-4, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout", 0.2, 0.7, step=0.1)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    
    #Model setup
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    for param in model.parameters(): param.requires_grad = False # Freeze
    for param in model.layer4.parameters(): param.requires_grad = True # Partial unfreeze
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(512, 30)
    )
    
    model = model.to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    # Optimizer
    optimizer = optim.Adam([
        {'params': model.module.layer4.parameters() if isinstance(model, nn.DataParallel) else model.layer4.parameters(), 'lr': lr_backbone},
        {'params': model.module.fc.parameters() if isinstance(model, nn.DataParallel) else model.fc.parameters(), 'lr': lr_head}
    ], weight_decay=weight_decay)
    
    criterion = nn.CrossEntropyLoss()
    
    # DataLoader, creates everytime the sampler
    train_sampler = BalancedBatchSampler(train_df, batch_size=CONFIG['batch_size'])
    train_loader = DataLoader(train_ds, batch_sampler=train_sampler, num_workers=CONFIG['num_workers'])
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'])

    best_val_acc = 0.0
    
    #Brief training loop
    for epoch in range(CONFIG['epochs_per_trial']):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        
        #Middle Report for pruning
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
        if accuracy > best_val_acc:
            best_val_acc = accuracy

    return best_val_acc

#Studio Start
log("\n Starting Optuna Optimization...")
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())

try:
    study.optimize(objective, n_trials=CONFIG['n_trials'])
    
    log("\n Optuna Finished")
    log(f"🏆 Best Value (Acc): {study.best_trial.value:.4f}")
    log("🏆 Best Params:")
    for key, value in study.best_trial.params.items():
        log(f"    {key}: {value}")

    # Saving parameters on JSON
    import json
    with open("best_hyperparameters.json", "w") as f:
        json.dump(study.best_trial.params, f)
    log("Parameters saved in 'best_hyperparameters.json'")

except Exception as e:
    log(f"Critical Error during Optuna: {e}")